In [1]:
import skimage
from bioio import BioImage
import numpy as np
from scipy import ndimage
import stackview

In [2]:
from atracks.analysis import pattern_template
from skimage.measure import regionprops, label

In [3]:
fname = "../../data/Movie S1 growth-cropped-350-449.tif"
#fname = "../../data/Movie S1 growth-cropped.tif"
#fname = "../../data/-11C_denoised-cropped-short.tif"
#fname = "../../data/-11C_denoised-cropped.tif"
imgh = BioImage(fname)
img = imgh.get_image_data().squeeze()
img.shape

(100, 233, 500)

In [4]:
stackview.slice(img[0], continuous_update=True)

In [5]:
from atracks.preprocess import preprocess 
from atracks.analysis import analyze, iterative_threshold, identify_lattice_holes, weighted_temporal_sum
from atracks.vis import voronois

# preprocess: remove noise and enhance objects
denoised, dog = preprocess(img)

# analyze: segment and filter objects
mask, f_mask, h_mask, final_stats, probs_img, points = analyze(dog)
print (final_stats["combined"])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  22 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.16719437679056498s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  44 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  62 out of 100 | elapsed:    0.5s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  84 out of 100 | elapsed:    0.5s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  95 out of 100 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.4s
[P

{'count': 44002, 'area_mean': 95.18312803963457, 'area_std': 145.6184103395051, 'area_median': 69.0, 'area_min': 20.0, 'area_max': 2118.0, 'perimeter_mean': 39.14141983070304, 'perimeter_std': 53.3385152547555, 'perimeter_median': 28.14213562373095, 'perimeter_min': 13.65685424949238, 'perimeter_max': 873.2874093090415, 'circularity_mean': 0.978924318770044, 'circularity_std': 0.23489266830662398, 'circularity_median': 1.0630912983608525, 'circularity_min': 0.03393740867863165, 'circularity_max': 1.430983102553298, 'bbox_width_mean': nan, 'bbox_width_std': nan, 'bbox_width_median': nan, 'bbox_width_min': nan, 'bbox_width_max': nan, 'bbox_height_mean': nan, 'bbox_height_std': nan, 'bbox_height_median': nan, 'bbox_height_min': nan, 'bbox_height_max': nan, 'bbox_area_mean': 186.40568610517704, 'bbox_area_std': 546.3937272302119, 'bbox_area_median': 90.0, 'bbox_area_min': 22.0, 'bbox_area_max': 11350.0, 'solidity_mean': 0.9306784906236293, 'solidity_std': 0.09767518496426407, 'solidity_med

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.010154008865356445s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  22 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.08188509941101074s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  60 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  95 out of 100 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1458568572998047s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  13 ta

target shape: (100, 233, 500), actual shape: (100, 233, 500)
{'count': 36720, 'area_mean': 67.56187363834422, 'area_std': 10.492916252338341, 'area_median': 68.0, 'area_min': 35.0, 'area_max': 103.0, 'perimeter_mean': 28.2517870321719, 'perimeter_std': 2.680638741342595, 'perimeter_median': 28.142135623730947, 'perimeter_min': 19.071067811865476, 'perimeter_max': 40.04163056034262, 'circularity_mean': 1.0617856202799747, 'circularity_std': 0.06970976204790892, 'circularity_median': 1.0780241689052945, 'circularity_min': 0.7973513457046634, 'circularity_max': 1.2438337708587477, 'bbox_width_mean': nan, 'bbox_width_std': nan, 'bbox_width_median': nan, 'bbox_width_min': nan, 'bbox_width_max': nan, 'bbox_height_mean': nan, 'bbox_height_std': nan, 'bbox_height_median': nan, 'bbox_height_min': nan, 'bbox_height_max': nan, 'bbox_area_mean': 90.06791938997821, 'bbox_area_std': 16.624389687751968, 'bbox_area_median': 90.0, 'bbox_area_min': 40.0, 'bbox_area_max': 182.0, 'solidity_mean': 0.961465

[Parallel(n_jobs=-1)]: Done  84 out of 100 | elapsed:    1.5s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  95 out of 100 | elapsed:    1.6s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.6s finished


In [6]:
from atracks.analysis import weighted_temporal_sum
dog_conf = weighted_temporal_sum(dog, decay=0.5, n=2)
f_mask_conf = weighted_temporal_sum(f_mask.astype(float), n=2)

In [7]:
min_val = np.min(dog)
mean_val = np.mean(dog)
max_val = np.max(dog)
print (min_val, mean_val, max_val)
th1 = 0.5 * (max_val-mean_val) + mean_val
th2 = 0.25 * (max_val-mean_val) + mean_val
th3 = 0.125 * (max_val-mean_val) + mean_val
print (th1, th2, th3)

0.0 0.2787064 1.0
0.63935316 0.4590298 0.3688681


In [8]:
# top 50%
binaries2, f_regions2 = iterative_threshold(dog, threshold=(th1, None), mask=h_mask, iterations=25, area=(15,85), solidity=(0.50,1.0))
f_mask2 = f_mask | (np.sum(binaries2, axis=0)>0) #(f_regions2 >0)
h_mask2 = identify_lattice_holes(f_mask2, radius=final_stats["combined"]["nn_min_dist_mean"] // 2)

# 25-50%
binaries3, f_regions3 = iterative_threshold(dog, threshold=(th2, th1), area=(10,90), solidity=(0.50,1.0), aspect_ratio=(0.2,1.0), mask=h_mask2, iterations=25)
f_mask3 = f_mask2 | (np.sum(binaries3, axis=0)>0) #(f_regions3 >0)
h_mask3 = identify_lattice_holes(f_mask3, radius=final_stats["combined"]["nn_min_dist_mean"] // 2)

# 12.5-25%
binaries4, f_regions4 = iterative_threshold(dog, threshold=(th3, th2), area=(10,90), solidity=(0.5,1.0), aspect_ratio=(0.3,1.0), mask=h_mask3, iterations=25)
f_mask4 = f_mask3 | (np.sum(binaries4, axis=0)>0) #(f_regions4 >0)
h_mask4 = identify_lattice_holes(f_mask4, radius=final_stats["combined"]["nn_min_dist_mean"] // 2)

In [9]:
import numpy as np
from skimage.filters import threshold_otsu, threshold_li
high_conf_mean=np.mean(f_mask | f_mask2, axis=0)
th = threshold_li(high_conf_mean)
high_conf = high_conf_mean > th
include = ~identify_lattice_holes(high_conf, radius=final_stats["combined"]["nn_min_dist_mean"] * 2).astype(bool)

from skimage.morphology import dilation, opening, disk  # noqa
footprint = disk(final_stats["combined"]["nn_min_dist_mean"] // 2)
include = dilation(opening(include, footprint), footprint)
#stackview.insight(include)

In [10]:
mask_1 = f_mask & include
mask_2 = (f_regions2 > 0) & include
mask_3 = (f_regions3 > 0) & include
mask_4 = (f_regions4 > 0) & include
mask_all = mask_1 | mask_2 | mask_3 | mask_4

In [11]:
points_layer, polygons = voronois(mask_all)

color_cycle: ['red', 'green', 'blue', 'yellow', 'purple', 'orange', 'brown', 'pink', 'gray', 'black']


# Animation

In [12]:
import napari
viewer = napari.Viewer()

In [ ]:
viewer.add_image(img)
viewer.add_image(dog)
#viewer.add_image(dog_conf)
#viewer.add_image(f_mask_conf)

viewer.add_points(points, name="Atoms from Points", face_color="pink", size=7)
#viewer.add_image(mask, colormap="green", opacity=0.2)
#viewer.add_image(f_mask, colormap="magenta", opacity=0.2)
#viewer.add_image(h_mask, colormap="orange", opacity=0.2)
viewer.add_image(mask_1, name="f_regions1 mask", colormap="green", opacity=0.2)
viewer.add_image(mask_2, name="f_regions2 mask", colormap="cyan", opacity=0.2)
viewer.add_image(mask_3, name="f_regions3 mask", colormap="magenta", opacity=0.2)
viewer.add_image(mask_4, name="f_regions4 mask", colormap="yellow", opacity=0.2)
viewer.add_image(mask_all, name="all mask", colormap="orange", opacity=0.2)
viewer.add_image(probs_img, colormap="plasma")

points_layer.size=7
viewer.layers.extend([points_layer])
viewer.add_shapes(polygons, name="Cells", edge_color="white", face_color=[0,0,0,0], opacity=0.3, shape_type="polygon")

In [ ]:
def export_animation(viewer, fname, fps=30):
    from napari_animation import Animation

    animation = Animation(viewer)
    
    viewer.dims.current_step = (0, 0, 0)
    animation.capture_keyframe()
    viewer.dims.current_step = (img.shape[0], 0, 0)
    animation.capture_keyframe(img.shape[0])
    
    animation.animate(fname, fps=fps)


In [ ]:
movie_file_stub = ".".join(fname.split(".")[:-1])
# dog
movie_file = movie_file_stub + "-enhanced.mp4"
for l in viewer.layers[2:6]:
    l.visible=False
for l in viewer.layers[-2:]:
    l.visible=False
export_animation(viewer, movie_file)

# segmentation
movie_file = movie_file_stub + "-segmentation.mp4"
for l in viewer.layers[2:6]:
    l.visible=True
for l in viewer.layers[-2:]:
    l.visible=False
export_animation(viewer, movie_file)

# tesselation
movie_file = movie_file_stub + "-tesselation.mp4"
for l in viewer.layers[2:6]:
    l.visible=False
for l in viewer.layers[-2:]:
    l.visible=True
export_animation(viewer, movie_file)